In [1]:
# What did I learned so far:
# 1) you do the best for the mean encoding of items in stock and shops open
#  and train a separate model for those
# 2) for the rest you force zero the discontinued items and closed shops
# 3) for the sparse shops and items you predict the mean of last tree month 

# Prepare data

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Load data
train_df = pd.read_csv("../sales_train.csv.gz")
test_df  = pd.read_csv("../test.csv.gz")

categories = pd.read_csv('../item_categories.csv')
items = pd.read_csv('../items.csv')

In [3]:
%store -r __trends_combined

In [4]:
%%time
__trends_combined

CPU times: user 1min 34s, sys: 48 s, total: 2min 22s
Wall time: 2min 30s


/usr/local/lib/python3.7/site-packages/pandas/core/reshape/merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:3947: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


[0, 1, 8, 11, 13, 17, 23, 27, 29, 30, 32, 33, 40, 43, 54]
15 out of 60 shops were not selling at least last 3 months
Sanity check: known item categories is 84 and total categories is 84
Total number of items is 22170, among those sold are 21807
Never sold are 363 items from 39 categories of 12 types and 32 subtypes
 (in total there are 20 types and 65 subtypes)
21807


/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:56: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


No sales for 19 categories, 4 types, and 15 subtypes
14627 out of 21807 items were not sold at least last 3 months
{27}
{16689, 15106, 20539, 5613}
CPU times: user 2min 21s, sys: 1min 15s, total: 3min 36s
Wall time: 3min 50s


# Sanity checks

## Compare with macro2

see if lags are the same

In [23]:
#lagged_sales_shops
#lagged_sales_cats
#lagged_sales_types
#lagged_sales_subtypes
#lagged_sales_items

all_data[['date_block_num',
          'shop_id',
          'item_id',
          'total_monthly_shop_sales_lag_1',
          'trg_shop_lag1']][(all_data.item_id==28)&(all_data.shop_id==1)]

# in reality, item_id was aggregated over so selecting one above doesn't affect lags

,date_block_num,shop_id,item_id,total_monthly_shop_sales_lag_1,trg_shop_lag1
8117,0,1,28,NaN,0.00000
373345,1,1,28,0.361306,1.91987


In [6]:
total_sales_shops[total_sales_shops.shop_id==1]

,shop_id,target_sum,target_len,trg_shop
1,1,6311.0,3282.0,1.922913


In [51]:
# difference comes only from clipping sales to <=20
print(sum(all_data[(all_data.shop_id==1)].target))
print(sum(train_df[train_df.shop_id==1].item_cnt_day))
print(sum(agg[agg.shop_id==1].target.clip(0,20)))

qwe = monthly[(monthly.shop_id==1)&(monthly.date_block_num==1)]

print("%d / %d = %f" % (sum(qwe.target),qwe.shape[0],sum(qwe.target)/qwe.shape[0]))

all_data.groupby(['shop_id','date_block_num']).agg({'target': [sum,len,np.mean]}).head()

6246.0
6311.0
6246.0
(1747, 4)
3364 / 1747 = 1.925587


target                  
                           sum     len      mean
shop_id date_block_num                          
0       0               5411.0  8115.0  0.666790
        1               5820.0  8168.0  0.712537
1       0               2932.0  8115.0  0.361306
        1               3314.0  8168.0  0.405730
2       0               1146.0  8115.0  0.141220

Everything squares up nicely above

In [ ]:
# mean encoding: average target, aggregated across a given category
#  averaging happens across all entries => no way to distinguish a closed shop against no sales
#  therefore, there is a potential problem with aggregating over the date_block_num

Now let's do the following:
* exclude closed shops from training
* augment model from macro2 with category/type/supertype

In [ ]:
#qwe